In [ ]:
# For loop predict using latest generate softmax
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import nibabel as nib
import numpy as np
from scipy import ndimage
import pandas as pd
import os
import SimpleITK as sitk
import pickle

def list_labels(directory):
    file_paths = []  # List to store the file paths
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths


def list_outputs(directory):
    file_paths = []  # List to store the file paths
    entries = os.listdir(directory)
    folds = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
    for f in folds:
        subfolder = directory + f + '/validation'
        for root, dirs, files in os.walk(subfolder):
            for file in files:
                file_path = os.path.join(root, file)
                if file.endswith('.nii.gz'):
                    file_paths.append(file_path)
    return file_paths


# instantiate the nnUNetPredictor
if __name__ == "__main__":
    nnUNet_results = 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV'
    nnUNet_raw = 'Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/MRF_to_nnunet/nnUNet_raw'
    predictor = nnUNetPredictor(tile_step_size=0.5,
            use_gaussian=True,
            use_mirroring=True,
            perform_everything_on_device=True,
            device=torch.device('cuda', 0),
            verbose=False,
            verbose_preprocessing=False,
            allow_tqdm=True)

    directory = join(nnUNet_results, 'Dataset022_T1wT1T2pt/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres')
    entries = os.listdir(directory)
    folds = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
    last_2 = [item[-2:] for item in folds]

    for ind, f in enumerate(last_2):
        if f[0] == '_':
            fold = int(f[-1])
        else:
            fold = int(f) 
        indir = join(nnUNet_raw, 'Dataset019_vbm/imagesTr')
        for root, dirs, files in os.walk(indir):
            flist = files
        
        try:
            predictor.initialize_from_trained_model_folder(
                    join(nnUNet_results, 'Dataset016_MRFz/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres'),
                    use_folds=(fold,),
                    checkpoint_name='checkpoint_best.pth')
        except:
           predictor.initialize_from_trained_model_folder(
                    join(nnUNet_results, 'Dataset016_MRFz/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres'),
                    use_folds=(fold,),
                    checkpoint_name='checkpoint_final.pth')
        outdir = join(nnUNet_results, 'Dataset016_MRFz/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres_softmax/')
        
        predictor.predict_from_files([[join(indir, flist[fold*8]), join(indir, flist[fold*8+1]),
                                       join(indir, flist[fold*8+2]), join(indir, flist[fold*8+3]),
                                       join(indir, flist[fold*8+4]), join(indir, flist[fold*8+5]),
                                       join(indir, flist[fold*8+6]), join(indir, flist[fold*8+7])]],
                                     [join(outdir, flist[fold*8][:-12] + '.nii.gz')],
                                     save_probabilities=True, overwrite=False,
                                     num_processes_preprocessing=2, num_processes_segmentation_export=2,
                                     folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

In [ ]:
# loop find lacked predictions and predict using best
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import nibabel as nib
import numpy as np
from scipy import ndimage
import pandas as pd
import os
import SimpleITK as sitk
import pickle

def list_labels(directory):
    file_paths = []  # List to store the file paths
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths


def list_outputs(directory):
    file_paths = []  # List to store the file paths
    entries = os.listdir(directory)
    folds = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
    for f in folds:
        subfolder = directory + f + '/validation'
        for root, dirs, files in os.walk(subfolder):
            for file in files:
                file_path = os.path.join(root, file)
                if file.endswith('.nii.gz'):
                    file_paths.append(file_path)
    return file_paths


# instantiate the nnUNetPredictor
if __name__ == "__main__":
    nnUNet_results = 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV'
    nnUNet_raw = 'Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/MRF_to_nnunet/nnUNet_raw'
    predictor = nnUNetPredictor(tile_step_size=0.5,
            use_gaussian=True,
            use_mirroring=True,
            perform_everything_on_device=True,
            device=torch.device('cuda', 0),
            verbose=False,
            verbose_preprocessing=False,
            allow_tqdm=True)

    directory = join(nnUNet_results, 'Dataset023_nosmooth/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres')
    entries = os.listdir(directory)
    folds = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
    last_2 = [item[-2:] for item in folds]

    for ind, f in enumerate(last_2):
        if f[0] == '_':
            fold = int(f[-1])
        else:
            fold = int(f) 
        indir = join(nnUNet_raw, 'Dataset023_nosmooth/imagesTr')
        for root, dirs, files in os.walk(indir):
            flist = files
      
        predictor.initialize_from_trained_model_folder(
                join(nnUNet_results, 'Dataset023_nosmooth/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres'),
                use_folds=(fold,),
                checkpoint_name='checkpoint_best.pth')
        outdir = join(nnUNet_results, 'Dataset023_nosmooth/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/' + folds[ind] + '/validation')
        # print(outdir)
        

        if os.path.exists(outdir) and os.path.isdir(outdir):
            continue
        else:
            # predictor.predict_from_files([[join(indir, flist[fold*6]), join(indir, flist[fold*6+1]),
            #                            join(indir, flist[fold*6+2]), join(indir, flist[fold*6+3]),
            #                            join(indir, flist[fold*6+4]), join(indir, flist[fold*6+5])]],
            #                          [join(outdir, flist[fold*6][:-12] + '.nii.gz')],
            #                          save_probabilities=False, overwrite=False,
            #                          num_processes_preprocessing=2, num_processes_segmentation_export=2,
            #                          folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)
            predictor.predict_from_files([[join(indir, flist[fold*8]), join(indir, flist[fold*8+1]),
                                       join(indir, flist[fold*8+2]), join(indir, flist[fold*8+3]),
                                       join(indir, flist[fold*8+4]), join(indir, flist[fold*8+5]),
                                       join(indir, flist[fold*8+6]), join(indir, flist[fold*8+7])]],
                                     [join(outdir, flist[fold*8][:-12] + '.nii.gz')],
                                     save_probabilities=False, overwrite=False,
                                     num_processes_preprocessing=2, num_processes_segmentation_export=2,
                                     folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

In [ ]:
import nibabel as nib
import numpy as np
from scipy import ndimage
import pandas as pd
import os
import SimpleITK as sitk
import pickle


def process_nifti_file(nifti_file_path, binary_label_path):
    img = nib.load(nifti_file_path)
    data = img.get_fdata()
    olwithin = 0
    label_img = nib.load(binary_label_path)
    label_data = label_img.get_fdata()
    # Count non-overlapping clusters
    overlap = (data.astype(int) & label_data.astype(int)).astype(bool)
    overlap_per = np.sum((data.astype(int) & (label_data > 0).astype(int)))/(np.sum(label_data.astype(int)))
    dsc = 2*np.sum((data.astype(int) & (label_data > 0).astype(int)))/(np.sum((label_data > 0).astype(int))+np.sum(data.astype(int)))
    has_overlap = np.any(overlap)

    structure = np.ones((3, 3, 3), dtype=int)
    labeled_array, num_features = ndimage.label(data > 0, structure=structure)
    
    non_overlapping_clusters = 0
    for cluster_id in range(1, num_features + 1):
        cluster_mask = labeled_array == cluster_id
        overlap = np.any(
            cluster_mask & (label_data > 0))  # Assuming nonzero values in label_data indicate labeled regions
        if not overlap:
            non_overlapping_clusters += 1
        else:
            olwithin = np.sum(cluster_mask & (label_data > 0).astype(int))/(np.sum(cluster_mask))
        
    return has_overlap, non_overlapping_clusters, data, overlap_per, olwithin, dsc


def list_labels(directory):
    file_paths = []  # List to store the file paths
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths


def list_outputs(directory):
    file_paths = []  # List to store the file paths
    entries = os.listdir(directory)
    folds = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
    for f in folds:
        subfolder = directory + f + '/validation'
        for root, dirs, files in os.walk(subfolder):
            for file in files:
                file_path = os.path.join(root, file)
                if file.endswith('.nii.gz') and not file.endswith('_softmax.nii.gz'):
                    file_paths.append(file_path)
    return file_paths

if __name__ == '__main__':
    nifti_file_paths = list_outputs(
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV\Dataset009_MRF_clin/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV\Dataset003_MRF_T1w/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV\Dataset008_MRF_agegender/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV\Dataset007_MRF_T1T2z/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset012_clint1pt/nnUNetTrainer_500epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset013_MRFzclin/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres_last/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset014_MRFzclin/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset014_MRFzclin/nnUNetTrainer_250epochs__postprocessed2/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset015_MRFz/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset016_MRFz/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_Spencer/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset017_noCSF/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset022_T1wT1T2pt/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset016_MRFz/nnUNetTrainer_1000ep__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset002_MRF_small/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')
        # 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/Dataset023_nosmooth/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres/')

    binary_label_paths = list_labels(
        'Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/MRF_to_nnunet/nnUNet_raw/Dataset016_MRFz/labelsTr')
    img = nib.load('Z:/Imaging/Multimodal/MRF/Peter/header.nii')


    last_4 = {item[-16:] for item in nifti_file_paths}
    print(last_4)
    binary_label_paths = [item for item in binary_label_paths if item[-16:] in last_4]

    nifti_file_paths = sorted(nifti_file_paths, key=lambda x: x[-16:])
    binary_label_paths = sorted(binary_label_paths, key=lambda x: x[-16:])
    # print(nifti_file_paths)
    results = []
    ol = 0
    FPperP = 0
    operls = []
    withinls = []
    dscs = []
    for nifti_file_path, binary_label_path in zip(nifti_file_paths, binary_label_paths):
        has_overlap, num_non_overlapping_clusters, newimg, oper, olwithin, dsc = process_nifti_file(nifti_file_path, binary_label_path)
        results.append([nifti_file_path[-17:], int(has_overlap), num_non_overlapping_clusters, oper, olwithin, dsc])
        if has_overlap:
            ol = ol + 1
            operls.append(oper)
            withinls.append(olwithin)
            dscs.append(dsc)
        FPperP += num_non_overlapping_clusters

    summary_df = pd.DataFrame(results, columns=["File", "Overlap", "# of FP Clusters", "overlap%", "withinlabel%", "dsc"])
    summary_df.to_excel('Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/OptimalPerformance.xlsx', index=False)
    # print('niftis:\n', len(nifti_file_paths))
    # print('labels:\n', len(binary_label_paths))
    print(summary_df)
    print("Total overlapped:\n", ol)
    print("FP per patient:\n", FPperP / len(nifti_file_paths))
    print("overlap per patient:\n", np.mean(np.array(operls)))
    print("std overlap:\n", np.std(np.array(operls)))
    print("ol within:\n", np.mean(np.array(withinls)))
    print("std ol within:\n", np.std(np.array(withinls)))
    print("dsc:\n", np.mean(np.array(dscs)))
    print("std dsc:\n", np.std(np.array(dscs)))

In [ ]:
import nibabel as nib
import numpy as np
from scipy import ndimage
import pandas as pd
import os
import SimpleITK as sitk
import pickle
from openpyxl import load_workbook

# 6 fold CV results check
def process_nifti_file(nifti_file_path, binary_label_path):
    img = nib.load(nifti_file_path)
    data = img.get_fdata()
    olwithin = 0
    label_img = nib.load(binary_label_path)
    label_data = label_img.get_fdata()
    # Count non-overlapping clusters
    overlap = (data.astype(int) & label_data.astype(int)).astype(bool)
    overlap_per = np.sum((data.astype(int) & (label_data > 0).astype(int)))/(np.sum(label_data.astype(int)))
    has_overlap = np.any(overlap)

    structure = np.ones((3, 3, 3), dtype=int)
    labeled_array, num_features = ndimage.label(data > 0, structure=structure)
    
    non_overlapping_clusters = 0
    for cluster_id in range(1, num_features + 1):
        cluster_mask = labeled_array == cluster_id
        overlap = np.any(
            cluster_mask & (label_data > 0))  # Assuming nonzero values in label_data indicate labeled regions
        if not overlap:
            non_overlapping_clusters += 1
        else:
            olwithin = np.sum(cluster_mask & (label_data > 0).astype(int))/(np.sum(cluster_mask))

    return has_overlap, non_overlapping_clusters, data, overlap_per, olwithin


def list_labels(directory):
    file_paths = []  # List to store the file paths
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths


def list_outputs(directory):
    file_paths = []  # List to store the file paths
    entries = os.listdir(directory)
    subfolder = directory
    for root, dirs, files in os.walk(subfolder):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.nii.gz') and not file.endswith('_softmax.nii.gz'):
                file_paths.append(file_path)
    return file_paths

if __name__ == '__main__':
    nifti_file_paths = list_outputs(
        'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/6foldLOOCV/Dataset036_6fold6/prediction/')

    binary_label_paths = list_labels(
        'Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/MRF_to_nnunet/nnUNet_raw/Dataset045_6fold20/labelsTs')
    img = nib.load('Z:/Imaging/Multimodal/MRF/Peter/header.nii')

    last_4 = {item[-16:] for item in nifti_file_paths}
    binary_label_paths = [item for item in binary_label_paths if item[-16:] in last_4]

    nifti_file_paths = sorted(nifti_file_paths, key=lambda x: x[-16:])
    binary_label_paths = sorted(binary_label_paths, key=lambda x: x[-16:])
    # print(nifti_file_paths)
    # print(binary_label_paths)
    results = []
    ol = 0
    FPperP = 0
    operls = []
    withinls = []
    for nifti_file_path, binary_label_path in zip(nifti_file_paths, binary_label_paths):
        has_overlap, num_non_overlapping_clusters, newimg, oper, olwithin = process_nifti_file(nifti_file_path, binary_label_path)
        results.append([nifti_file_path[-17:], int(has_overlap), num_non_overlapping_clusters, oper, olwithin])
        if has_overlap:
            ol = ol + 1
            operls.append(oper)
            withinls.append(olwithin)
        FPperP += num_non_overlapping_clusters

    summary_df = pd.DataFrame(results, columns=["File", "Overlap", "# of FP Clusters", "overlap%", "withinlabel%"])
    excel_path = 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/6foldsLOOCVtest.xlsx'
    book = load_workbook(excel_path)
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a' if book else 'w') as writer:
        summary_df.to_excel(writer, sheet_name='test6', index=False)
    print('niftis:\n', len(nifti_file_paths))
    print('labels:\n', len(binary_label_paths))
    print(summary_df)
    print("Total overlapped:\n", ol)
    print("FP per patient:\n", FPperP / len(nifti_file_paths))
    print("overlap per patient:\n", np.mean(np.array(operls)))
    print("std overlap:\n", np.std(np.array(operls)))
    print("ol within:\n", np.mean(np.array(withinls)))
    print("std ol within:\n", np.std(np.array(withinls)))

In [ ]:
import nibabel as nib
import numpy as np
from scipy import ndimage
import pandas as pd
import os
import SimpleITK as sitk
import pickle

def process_nifti_file(nifti_file_path, binary_label_path):
    img = nib.load(nifti_file_path)
    data = img.get_fdata()
    olwithin = 0
    label_img = nib.load(binary_label_path)
    label_data = label_img.get_fdata()
    # Count non-overlapping clusters
    overlap = ((data.astype(float)>= 0.5) & label_data.astype(int)).astype(bool)
    overlap_per = np.sum((data.astype(float)>= 0.5) & (label_data > 0).astype(int))/(np.sum(label_data.astype(int)))
    has_overlap = np.any(overlap)
    structure = np.ones((3, 3, 3), dtype=int)
    labeled_array, num_features = ndimage.label(data >= 0.5, structure=structure)
    
    non_overlapping_clusters = 0
    for cluster_id in range(1, num_features + 1):
        cluster_mask = labeled_array == cluster_id
        overlap = np.any(cluster_mask & (label_data > 0.5))  # Assuming nonzero values in label_data indicate labeled regions
        if not overlap:
            non_overlapping_clusters += 1
        else:
            olwithin = np.sum(cluster_mask & (label_data > 0).astype(int))/(np.sum(cluster_mask))

    return has_overlap, non_overlapping_clusters, data, overlap_per, olwithin
    # return has_overlap, non_overlapping_clusters, data
    
if __name__ == '__main__':
    subjID=[(studyID)]
    nifti_file_paths = []
    binary_label_paths = []
    for s in subjID:
        nlp = "Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/" + s + "/MRF_VBM/MNI_lesionprob_ANN_fn.nii"
        blp = "Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/" + s + "/MRF_VBM/MNI_ROI_final.nii"
        nifti_file_paths.append(nlp)
        binary_label_paths.append(blp)

    img = nib.load('Z:/Imaging/Multimodal/MRF/Peter/header.nii')

    results = []
    ol = 0
    FPperP = 0
    operls = []
    withinls = []
    for nifti_file_path, binary_label_path in zip(nifti_file_paths, binary_label_paths):
        has_overlap, num_non_overlapping_clusters, newimg, oper, olwithin = process_nifti_file(nifti_file_path, binary_label_path)
        results.append([nifti_file_path[48:54], int(has_overlap), num_non_overlapping_clusters, oper, olwithin])
        if has_overlap:
            ol = ol + 1
            operls.append(oper)
            withinls.append(olwithin)
        FPperP += num_non_overlapping_clusters

    summary_df = pd.DataFrame(results, columns=["File", "Overlap", "# of FP Clusters", "overlap%", "withinlabel%"])
    # summary_df = pd.DataFrame(results, columns=["File", "Overlap", "# of FP Clusters"])
    summary_df.to_excel('Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV/MAP18.xlsx', index=False)
    print('niftis:\n', len(nifti_file_paths))
    print('labels:\n', len(binary_label_paths))
    print(summary_df)
    print("Total overlapped:\n", ol)
    print("FP per patient:\n", FPperP / len(nifti_file_paths))
    print("overlap per patient:\n", np.mean(np.array(operls)))
    print("std overlap:\n", np.std(np.array(operls)))
    print("ol within:\n", np.mean(np.array(withinls)))
    print("std ol within:\n", np.std(np.array(withinls)))

In [ ]:
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
nnUNet_results = 'Z:/Imaging/Multimodal/MRF/Peter/nnUNet_CV'
nnUNet_raw = 'Z:/Imaging/Multimodal/MRF/Recon_MRF_3T/Patients/MRF_to_nnunet/nnUNet_raw'
predictor = nnUNetPredictor(tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_device=True,
        device=torch.device('cuda', 0),
        verbose=False,
        verbose_preprocessing=False,
        allow_tqdm=True)
# initializes the network architecture, loads the checkpoint
predictor.initialize_from_trained_model_folder(
        join(nnUNet_results, 'Dataset009_MRF_clin/nnUNetTrainer_250epochs__nnUNetPlans__3d_fullres'),
        use_folds=(21,),
        checkpoint_name='checkpoint_best.pth')

In [ ]:
predictor.__getattribute__()

In [ ]:
from torchvision.models import resnet50
model = resnet50(pretrained=True)

In [ ]:
dir(model)

In [ ]:
model.state_dict

In [ ]:
predictor.network.load_state_dict

In [ ]:
predictor.list_of_parameters

In [ ]:
predictor.network.modules

In [ ]:
import argparse
import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models
import torch.backends.cudnn as cudnn
import os

In [ ]:
dir(predictor.network)

In [ ]:
for param in predictor.network.parameters():
    param.requires_grad = False

#set model in eval mode
predictor.network.eval()
#transoform input PIL image to torch.Tensor and normalize
input = transform(img)
input.unsqueeze_(0)

#we want to calculate gradient of higest score w.r.t. input
#so set requires_grad to True for input 
input.requires_grad = True
#forward pass to calculate predictions
preds = predictor.network(input)
score, indices = torch.max(preds, 1)
#backward pass to get gradients of score predicted class w.r.t. input image
predictor.network.backward()
#get max along channel axis
slc, _ = torch.max(torch.abs(input.grad[0]), dim=0)
#normalize to [0..1]
slc = (slc - slc.min())/(slc.max()-slc.min())

#apply inverse transform on image
with torch.no_grad():
    input_img = inv_normalize(input[0])
#plot image and its saleincy map
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(np.transpose(input_img.detach().numpy(), (1, 2, 0)))
plt.xticks([])
plt.yticks([])
plt.subplot(1, 2, 2)
plt.imshow(slc.numpy(), cmap=plt.cm.hot)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
predictor.network(torch.rand((8, 100, 100, 100)))

In [ ]:
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
img, props = SimpleITKIO().read_images([join(nnUNet_raw, 'Dataset003_Liver/imagesTr/liver_63_0000.nii.gz')])